In [5]:
# coding=utf-8
from keras.models import Sequential, load_model
from keras.layers import Dense
import numpy as np
import random
from collections import deque
import sys                                   # Pro pripojeni knihovny Open AI Gym
sys.path.append('/home/xbucha02/libraries')  # Pripojeni knihovny Open AI Gym
import gym
import numpy as np
from keras.models import save_model

BATCH_SIZE = 64
GAMMA = 0.9
EPSILON = 0.5


class DQN:
    def __init__(self, env):
        self.replay = deque(maxlen=10000)
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.epsilon = EPSILON

        self.create_model()

    def create_model(self):
        self.model = Sequential()
        self.model.add(Dense(20, activation='relu', input_shape=(self.state_dim,)))
        self.model.add(Dense(20, activation='relu'))
        self.model.add(Dense(self.action_dim, activation='linear'))
        self.model.compile(optimizer='Adam', loss='mse')

    def perceive(self, state, action, reward, next_state, done):
        self.replay.append((state, action, reward, next_state, done))
        if len(self.replay) > BATCH_SIZE:
            self.train_Q_network()

    def train_Q_network(self):
        minbatch = random.sample(self.replay, BATCH_SIZE)
        state_batch = np.vstack([i[0] for i in minbatch])
        action_batch = [i[1] for i in minbatch]
        reward_batch = [i[2] for i in minbatch]
        next_state_batch = np.vstack([i[3] for i in minbatch])
        y_batch = []
        Q_value_batch = self.model.predict(next_state_batch)

        for i in range(BATCH_SIZE):
            if minbatch[i][4]:
                reward = reward_batch[i]
            else:
                reward = reward_batch[i] + GAMMA * np.max(Q_value_batch[i])
            target = [0 for _ in range(self.action_dim)]
            target[action_batch[i]] = reward
            y_batch.append(target)
        print("state_batch:{} y_batch: {}" .format(state_batch, y_batch))
        self.model.train_on_batch(state_batch, y_batch)

    def e_action(self, state):
        Q_value = self.model.predict(state)
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        else:
            return np.argmax(Q_value)
        self.epsilon -= (EPSILON - 0.01) / 10000

    def action(self, state):
        return np.argmax(self.model.predict(state))

    def load(self, model):
        self.model = load_model(model)



env = gym.make('CartPole-v0')
agent = DQN(env)
flag = 0
#agent.load('2.h5')

for i in range(10000):
    state = env.reset()
    for s in range(400):
        state = np.reshape(state, (1, agent.state_dim))
        action = agent.e_action(state)
        next_state, reward, done, _ = env.step(action)
        reward = -1 if done else 0.1
        agent.perceive(state, action, reward, next_state, done)
        print("State: {}" .format(state))
        state = next_state
        if done:
            break

    if i % 100 == 0:
        t_reward = 0
        for j in range(10):
            state = env.reset()
            while True:
                state = np.reshape(state, (1, agent.state_dim))
                action = agent.action(state)
                #env.render()
                state, reward, done, _ = env.step(action)
                t_reward += reward
                if done:
                    break
        print(i, t_reward / 10, len(agent.replay))
        if t_reward / 10 >= 250:
            break
    if t_reward / 10 > flag:
        flag = t_reward / 10
        save_model(agent.model, 'cartpole.h5')

State: [[ 0.02535736  0.04298877 -0.04530947  0.00893119]]
State: [[ 0.02621714 -0.15145508 -0.04513085  0.28698121]]
State: [[ 0.02318804 -0.34590533 -0.03939122  0.56509563]]
State: [[ 0.01626993 -0.15025351 -0.02808931  0.26026753]]
State: [[ 0.01326486 -0.34496344 -0.02288396  0.54396006]]
State: [[ 0.00636559 -0.14952751 -0.01200476  0.24415562]]
State: [[ 0.00337504 -0.34447596 -0.00712165  0.5330279 ]]
State: [[-0.00351448 -0.53949703  0.00353891  0.82345834]]
State: [[-0.01430442 -0.73466721  0.02000808  1.11725222]]
State: [[-0.02899776 -0.93004594  0.04235312  1.41614364]]
State: [[-0.04759868 -1.12566607  0.070676    1.7217588 ]]
State: [[-0.070112   -1.32152255  0.10511117  2.03557152]]
State: [[-0.09654245 -1.12762901  0.1458226   1.7771822 ]]
State: [[-0.11909503 -1.32406154  0.18136625  2.11142387]]
(0, 8.8, 14)
State: [[-0.0151036  -0.01691346  0.00407434 -0.04501338]]
State: [[-0.01544187 -0.2120936   0.00317407  0.24895225]]
State: [[-0.01968374 -0.01701712  0.0081531

State: [[ 0.00226875 -0.35470207  0.04864675  0.57204083]]
state_batch:[[ -1.33729569e-01  -6.09894426e-01   1.69235494e-01   1.01382933e+00]
 [ -3.72666305e-02  -1.88297109e-01  -4.03140458e-02   2.46742826e-01]
 [ -3.51447701e-03  -5.39497026e-01   3.53891265e-03   8.23458338e-01]
 [ -7.01120020e-02  -1.32152255e+00   1.05111173e-01   2.03557152e+00]
 [ -9.26428355e-02  -5.62015156e-01   1.36662118e-01   1.05242296e+00]
 [ -9.65424530e-02  -1.12762901e+00   1.45822603e-01   1.77718220e+00]
 [ -8.53341498e-02  -3.65434282e-01   1.22183773e-01   7.23917249e-01]
 [ -5.63512875e-02  -3.60687510e-01   7.00328268e-02   6.15672827e-01]
 [  5.44853600e-03  -1.58989390e-01   4.33264805e-02   2.66013302e-01]
 [ -1.33342295e-01  -5.80859704e-01   9.42541840e-02   8.86296710e-01]
 [ -2.92062825e-02  -2.12437999e-01   1.59392635e-02   2.56572455e-01]
 [ -2.00240842e-02   1.77986976e-01   7.29856037e-03  -3.32827254e-01]
 [ -9.48613574e-02  -5.77545409e-01   3.92708497e-02   8.10328994e-01]
 [ -1.

State: [[ 0.00718706  0.40239509 -0.03747338 -0.61288516]]
state_batch:[[ -2.03645331e-02   2.04764356e-01  -2.60952549e-03  -2.64556658e-01]
 [ -1.35018431e-02  -1.83460795e-01  -6.32974921e-03   2.76285330e-01]
 [ -1.60502011e-01  -5.83614662e-01   1.36121303e-01   9.51463533e-01]
 [ -7.67541514e-03   1.02549379e-02  -1.77455646e-02   1.46150340e-02]
 [ -8.53341498e-02  -3.65434282e-01   1.22183773e-01   7.23917249e-01]
 [ -6.00553856e-02  -7.51026077e-01   1.52311269e-01   1.30370842e+00]
 [  6.36559233e-03  -1.49527511e-01  -1.20047576e-02   2.44155615e-01]
 [ -1.17671385e-01  -8.02909234e-01   1.44072675e-01   1.25814095e+00]
 [  4.71380182e-03   3.67367090e-02   4.41316905e-02  -4.02605001e-02]
 [ -1.22578724e-01  -3.73357108e-01   1.96690315e-01   9.09896866e-01]
 [ -2.89977617e-02  -9.30045943e-01   4.23531238e-02   1.41614364e+00]
 [ -5.36746241e-02  -7.98740388e-01   4.92252376e-02   1.15610449e+00]
 [ -4.16107195e-02  -6.03195234e-01   3.21555117e-02   8.53486297e-01]
 [  7.

State: [[ 0.01214177  0.02795285 -0.03932375 -0.06235365]]
state_batch:[[ -2.46514390e-02   2.04748522e-01   2.09204665e-03  -2.64210326e-01]
 [  7.55555937e-02   2.15070337e-01  -1.48347591e-01  -4.96951551e-01]
 [ -1.71710590e-02   1.17508884e-02  -8.04042616e-04  -1.83872627e-02]
 [ -1.33342295e-01  -5.80859704e-01   9.42541840e-02   8.86296710e-01]
 [ -4.75789427e-03   5.97247618e-01  -1.94857834e-02  -8.99380011e-01]
 [ -2.00199521e-02   2.21975633e-01   8.75043101e-03  -3.30768298e-01]
 [  7.98570004e-02   2.23173329e-02  -1.58286622e-01  -2.54452051e-01]
 [  9.74934197e-02   4.21115782e-01  -2.07847061e-01  -1.04595377e+00]
 [ -9.77417024e-02  -9.96484107e-01   1.13807924e-01   1.51323752e+00]
 [  1.32648612e-02  -3.44963444e-01  -2.28839587e-02   5.43960056e-01]
 [ -7.67541514e-03   1.02549379e-02  -1.77455646e-02   1.46150340e-02]
 [ -1.06412266e-01  -3.82982873e-01   5.54774296e-02   5.30252864e-01]
 [  3.94528824e-02   7.95451801e-01  -8.81807443e-02  -1.26365987e+00]
 [ -1.

State: [[ 0.02280146 -0.24127967  0.00712731  0.2796618 ]]
state_batch:[[ -4.86889886e-02  -5.77427531e-01  -2.48503460e-02   8.07770280e-01]
 [ -1.02839152e-02   1.12421221e-02  -1.15338834e-02  -7.16457225e-03]
 [  5.44853600e-03  -1.58989390e-01   4.33264805e-02   2.66013302e-01]
 [ -2.48444106e-02   9.64857850e-03   1.53227148e-03   2.79887589e-02]
 [  2.34005154e-02  -1.53806361e-01  -9.93207982e-02  -6.46859080e-02]
 [  2.74634770e-02   5.99470269e-01  -6.91793800e-02  -9.50068215e-01]
 [ -1.62692459e-02   9.67974826e-03  -7.90065864e-03   2.73020583e-02]
 [  1.77550139e-02  -1.65310323e-01  -4.96656219e-02   1.89629034e-01]
 [ -1.33729569e-01  -6.09894426e-01   1.69235494e-01   1.01382933e+00]
 [ -2.92062825e-02  -2.12437999e-01   1.59392635e-02   2.56572455e-01]
 [ -7.50759072e-02  -9.47715880e-01   1.78385437e-01   1.63993410e+00]
 [  8.03033471e-02   2.19303539e-01  -1.63375663e-01  -5.92579702e-01]
 [  6.57265484e-03   3.40051293e-02  -4.62989359e-02  -1.96007544e-01]
 [ -1.

...

State: [[-0.04607762  0.18699501 -0.00261694 -0.28400333]]
state_batch:[[  1.13002262e-01  -5.36958214e-01  -1.97464657e-01   1.24704811e-01]
 [ -5.63833623e-03   2.68353526e-02  -1.23731007e-02  -3.76605270e-02]
 [  6.92178391e-02   1.49093190e-02  -2.82964752e-02  -1.71266033e-02]
 [ -1.69360412e-02   2.06884361e-01  -1.17178787e-03  -3.11323766e-01]
 [  1.06598322e-01   2.82822300e-02  -1.11440096e-01  -3.13950846e-01]
 [  2.92725975e-02   1.83287001e-01   1.59511464e-02  -2.65418270e-01]
 [ -2.63218093e-02   2.23189563e-01   2.36645228e-02  -4.09681381e-01]
 [  1.22183144e-01   3.91467736e-02  -1.77689929e-01  -5.60040893e-01]
 [ -2.46514390e-02   2.04748522e-01   2.09204665e-03  -2.64210326e-01]
 [ -2.92062825e-02  -2.12437999e-01   1.59392635e-02   2.56572455e-01]
 [ -2.15571499e-01  -1.29813155e+00   6.70537254e-02   8.68822221e-01]
 [ -1.43044175e-02  -7.34667210e-01   2.00080794e-02   1.11725222e+00]
 [ -2.41471432e-02   2.91480777e-02   2.57868422e-02  -1.40726215e-01]
 [  4.

state_batch:[[  1.22183144e-01   3.91467736e-02  -1.77689929e-01  -5.60040893e-01]
 [  2.60488537e-02  -1.10101365e-02   2.08603950e-02   9.11685736e-03]
 [  1.17383151e-02   6.16163617e-01  -5.35267611e-02  -1.05611473e+00]
 [ -4.11652705e-02  -9.44505758e-01   1.21186111e-01   1.55625789e+00]
 [  9.80820337e-03  -1.61777427e-01  -4.85305943e-02   1.11582919e-01]
 [  2.58286510e-02   1.83806532e-01   2.10427322e-02  -2.76912055e-01]
 [ -1.51966896e-01  -1.68757938e+00   1.54029626e-02   1.43518407e+00]
 [  3.76240530e-02   2.19404086e-01  -4.34062479e-02  -3.26296406e-01]
 [  2.74634770e-02   5.99470269e-01  -6.91793800e-02  -9.50068215e-01]
 [ -1.43044175e-02  -7.34667210e-01   2.00080794e-02   1.11725222e+00]
 [  2.29765627e-02   1.45451507e-01  -5.80967287e-03  -2.28218341e-01]
 [ -3.34550425e-02  -4.07783849e-01   2.10707126e-02   5.54239954e-01]
 [  4.82885766e-02   2.09107649e-01   1.05608348e-03  -2.89429750e-01]
 [ -2.42838945e-02   1.90576171e-01  -4.29284430e-02  -3.63282942

State: [[ 0.04704312  0.22206977 -0.07956251 -0.43565529]]
state_batch:[[  1.44488075e-02  -3.59687855e-01  -4.58730412e-02   4.66240115e-01]
 [ -1.11262015e-01  -5.65835446e-01   1.73821975e-01   1.14341697e+00]
 [ -9.65424530e-02  -1.12762901e+00   1.45822603e-01   1.77718220e+00]
 [ -1.09473137e-02  -1.84198102e-01  -1.13501154e-02   2.92568710e-01]
 [  1.12339209e-01   4.06110165e-01  -1.97278252e-01  -1.18966446e+00]
 [  3.08443466e-02   2.49597621e-01  -1.75446249e-01  -9.53255417e-01]
 [  1.00123061e-01   4.01760779e-01  -1.58653364e-01  -1.08511825e+00]
 [ -4.28525445e-02  -1.88438159e+00  -8.38888660e-02   1.77416702e+00]
 [  1.09054203e-01   4.24722999e-01  -1.40871133e-01  -1.04314560e+00]
 [  1.81567363e-02   4.55223680e-02  -1.10906859e-01  -4.52443928e-01]
 [ -5.63512875e-02  -3.60687510e-01   7.00328268e-02   6.15672827e-01]
 [ -1.79985527e-01  -1.97890565e-01   1.69239736e-01   4.64355203e-01]
 [  2.37225017e-02  -4.60518961e-02   7.43446325e-03  -1.53574874e-02]
 [  7.

state_batch:[[  1.04492267e-01   2.28096799e-01  -1.26602231e-01  -7.13445097e-01]
 [  5.12678534e-02  -2.33065155e-01  -7.96641044e-02   9.87960600e-02]
 [  7.09349062e-03   4.12616049e-01  -1.85827737e-03  -5.76587400e-01]
 [  4.34630385e-02   3.43276041e-01  -4.27532463e-02  -5.80920254e-01]
 [ -2.35641816e-02  -1.66333523e-01   2.29723179e-02   1.59979192e-01]
 [  2.76049700e-02  -8.51589555e-03   2.39391620e-02  -4.59205970e-02]
 [  2.40615874e-02   8.11952529e-01  -7.46490558e-02  -1.36510684e+00]
 [ -1.11624781e-02   1.07582188e-02  -1.14203315e-02   3.51080269e-03]
 [  1.62699313e-02  -1.50253505e-01  -2.80893094e-02   2.60267532e-01]
 [ -4.52500097e-02  -5.55063891e-01   5.22028358e-02   8.91499548e-01]
 [ -1.17671385e-01  -8.02909234e-01   1.44072675e-01   1.25814095e+00]
 [ -3.09516458e-02  -7.68386421e-03  -2.55962689e-02  -1.09500848e-03]
 [  3.16916592e-02  -4.84445750e-02  -2.15056945e-02   3.74313113e-02]
 [  1.09054203e-01   4.24722999e-01  -1.40871133e-01  -1.04314560

KeyboardInterrupt: 